# Bostrom Simulation
---

In [1]:
import time
import itertools

# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd
import matplotlib.pyplot as plt

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

from collections import Counter
from cadCAD import configs

# custom functions modules
from utils.partial_state_update_blocks import partial_state_update_blocks
# from utils.plots import df_preparator, boot_supply_plot
from utils.plots import *

## Initial state

In [2]:
sim_period = 10                   # amount of years for simulating
timesteps_per_year = 365        # units per year (days)
agents_count_init = 750
capitalization_per_agent_init = 1

initial_state = {
    'boot_inflation_rate': 0.10, # initial inflation rate
    'boot_supply': 1_000_000_000_000_000, # initial boot supply
    'boot_liquid_supply': 290_000_000_000_000, # initial boot liquid supply
    'boot_frozen_supply': 700_000_000_000_000, # initial boot frozen(on a gift contract) supply
    'boot_bonded_supply': 10_000_000_000_000, # initial boot bonded supply
    'boot_claimed_supply': 0, # initial boot claimed supply (from a gift contract)
    'boot_to_distribution_supply': 0, # initial boot to distribution supply (transferred to gift claimer)
    'hydrogen_supply': 10_000_000_000_000, # initial hydrogen supply (=initial boot bonded supply)
    'agents_count': agents_count_init, # initial count of active agents
    'capitalization_per_agent': capitalization_per_agent_init, # initial capitalization per agent in eth
    'capitalization_in_eth': agents_count_init * capitalization_per_agent_init, # initial cap in eth
    'cyberlinks_per_day': 0, # initial amount of cyberlinks per day
    'validator_revenue_gboot': 0, # initial validator revenue in gboot
    'timestep_provision_boot': 0, # initial timestep provision in gbott
    'gboot_price': agents_count_init * capitalization_per_agent_init / 1_000_000, # 10^6 = Total supply 10^15 / Gboot 10^9
    'cyberlinks_count': 0, # initial amount of cyberlinks
    'ampere_supply': 0, # initial amount of ampere resource token
    'volt_supply': 0, # initial amount of volt resource token
    'investmint_max_period': 90, # initial max period of investminting in days
    'ampere_mint_rate': 1, # initial rate if ampere minting rate
    'volt_mint_rate': 1, # initial rate if volt minting rate
    'gpu_memory_usage': 0, # initial gpu memory usage bytes
    'ampere_volt_ratio': 1, # initial ration between ampere and volt
    'volt_liquid_supply': 0, # initial volt liquid supply
    'ampere_liquid_supply': 0, # initial ampere liquid supply
    'volt_minted_amount': 0, # initial ampere minted amount
    'ampere_minted_amount': 0 # initial ampere minted amount
}

## Params for simulating

In [3]:
system_params = {
    # timesteps params
    'timesteps_per_year': [timesteps_per_year], # units per year(days)
    
    # boot inflation params
    'boot_inflation_rate_change_annual': [0.07], # maximum inflation rate change per year
    'boot_inflation_rate_min': [0.05], # minimum percent of the inflation
    'boot_inflation_rate_max': [0.20], # maximum percent of the inflation
    'boot_bonded_share_target': [0.70], # desireable ratio between vested tokens and tokens supply
    
#     # boot bonding amount params
#     'boot_bonding_share': [0.70],
    
    # gift claim dynamics params
    'days_for_gift_activation': [30], # 
    'claimed_at_activation_share': [0.85], 
    'days_for_gift_full_claim': [150],
    'boot_gift_amount_init': [700_000_000_000_000], # boots amount on a gift contract
    'liquid_boot_supply_share': [0.25], # desireble liquid boot supply share
    
    # understanding network effects params
    'agents_count_at_activation': [100_000], # amount of agents to gift release
    'start_capitalization_per_agent': [1], # initial capitalization per agent eth
#     'capitalization_per_agent': [1],
    
    # predicting volts demand params
    'cyberlinks_transactions_coeff': [15], # spped coefficient of cyberlinks txs growing
    'extra_links': [0], # extra links txs amount
    'guaranteed_links': [0], # guaranteed links txs amount
    
    # amper and volt minting params
    'ampere_base_halving_period': [365 * 3 / 2], # period of mint rate halving for Ampere mint rate
    'volt_base_halving_period': [365 * 3 / 2], # period of mint rate halving for Volt mint rate
    'ampere_mint_rate_init': [1], # initial mint rate for Amperes
    'volt_mint_rate_init': [1], # initial mint rate for Voltes
    'ampere_base_investmint_amount': [1_000_000_000], # initial cost for resource token in 1 cycle
    'volt_base_investmint_amount': [1_000_000_000], # initial cost for resource token in 1 cycle
    'ampere_base_investmint_period': [30], # base investmint amount for ampere days
    'volt_base_investmint_period': [30], # base investmint amount for volt days
    'investmint_max_period_init': [90], # initial value for
    'horizont_period_init': [365], # period for max investmint period doubling days
    
    # Investments into infrastructure params
    'validator_commission': [0.1], #average validator comission
    'max_validator_count': [92], # max amount of validators
    
    # Particles util
    'particle_per_cyberlink': [0.1] # amount of particle per cyberlink
}

## Context preparation

In [4]:
del configs[:]
experiment = Experiment()

sim_config = config_sim({
    'N': 1,
    'T': range(int(math.ceil(sim_period * timesteps_per_year))),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## Simulation

In [5]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()
simulation_result_df = pd.DataFrame(raw_result)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (3650, 28, 1, 27)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.51s


## Simulation results

In [ ]:
boot_supply_plot(simulation_result_df)
hydrogen_supply_plot(simulation_result_df)

In [ ]:
agents_count_plot(simulation_result_df, figsize=(16,5))
capitalization_plot(simulation_result_df, figsize=(16,5))
gboot_price_plot(simulation_result_df, figsize=(16,5))

In [ ]:
cyberlinks_per_day_plot(simulation_result_df)
cyberlinks_count_plot(simulation_result_df)

In [ ]:
volt_and_ampere_supply_plot(simulation_result_df, figsize=(16,5))
mint_rate_plot(simulation_result_df, figsize=(16,5))
minted_volt_ampere_plot(simulation_result_df, figsize=(16,5))

In [ ]:
gpu_memory_usage_plot(simulation_result_df)